
<div style="text-align: center; padding: 30px 10px;">

<h2 style="font-size: 30px; margin-top: 5px;">
Лабораторная работа 1. О смысле жизни.
</h2>

<hr style="width: 60%; border: 1px solid #10069f; margin: 25px auto;">

</div>

Если Вы видите <b><font color="#FF69B4"> Ваш ответ здесь </font></b> - надо поменять текст внутри блока на свой текстовый ответ на вопрос.


ВАЖНО! Здесь и далее от вас ожидается чистый код, близкий к тому, что нужно в индустрии. В контексте данной лабораторной работы ожидаются корректные названия у полей, методов и переменных, которые вы вводите; корректная инкапсуляция - нижнее подчеркивание или два перед названием метода или атрибута. Для корректной инкапсуляции можно (и даже нужно) добавлять нижние подчеркивания даже к названиям методов/атрибутов из условий задач :)

# 1. Абстракции (3 балла)

Напиши абстрактный класс `Organism`. От него будут наследоваться другие классы.

В нём должен быть:

Concrete Methods:

- `property` - `alive`, который проверяет, что текущий возраст не больше, чем максимальный и энергия не отрицательная. При попытке изменения (сеттер), нужно: если статус "живости" не изменяется, то ничего не меняем; если мёртвый -> живой, то возраст - максимальный - 1, энергия - 1; если живой -> мёртвый, то возраст делаем максимальным + 1.
- Мэджик метод, позволяющий использовать экземпляры в if-конструкциях (мэджик метод `__bool__`, связанный с `alive`).
- Инициализатор (`__init__`), который проставляет атрибуты экземпляра: размер популяции `population_size = 0`, текущий возраст `current_age = 0`, `current_energy = 1` и `id` в зависимости от текущей глобальной переменной `ID_COUNTER`. Последнюю надо увеличить на 1, чтобы последующие ID отличались.
- Метод класса (`classmethod`) `spawn`, принимающий на вход `population_size`, `current_age` и фиксирующий их для объекта.
- Метод `reproduce`, который возвращает новый объект данного класса, в котором `current_age = 0`, `current_energy` - меньше в `REPRODUCE_NEW_COEF` раз, округлённых вниз; population_size - меньше в `REPRODUCE_POPULATION_COEF` раз, округлённых вниз. Кроме того, энергия изначального объекта понижается в `REPRODUCE_OLD_COEF` раз и логгирует в поток вывода сообщение с ID тех, кто размножился и тех, кто появился.


Abstract Methods:

- Мэджик метод, отвечающий за репрезентацию объекта (str/repr)
- Метод `replenish_energy`, отвечающий за пополнение энергии



In [ ]:
from abc import ABC, abstractmethod
import math

ID_COUNTER = 0
REPRODUCE_NEW_COEF = 2
REPRODUCE_POPULATION_COEF = 2
REPRODUCE_OLD_COEF = 2

class Organism(ABC):
    def __init__(self):
        global ID_COUNTER
        self._population_size = 0
        self._current_age = 0
        self._current_energy = 1
        self._id = ID_COUNTER
        ID_COUNTER += 1
        self._max_age = None

    @property
    def alive(self):
        return self._current_age <= self._max_age and self._current_energy > 0

    @alive.setter
    def alive(self, value):
        if bool(self.alive) == bool(value):
            return

        if value and not self.alive:
            self._current_age = self._max_age - 1
            self._current_energy = 1
        elif not value and self.alive:
            self._current_age = self._max_age + 1

    def __bool__(self):
        return self.alive

    @classmethod
    def spawn(cls, population_size, current_age):
        obj = cls()
        obj._population_size = population_size
        obj._current_age = current_age
        return obj

    def reproduce(self):
        if not self.alive:
            print(f"Организм {self._id} мертв и не может размножаться.")
            return None

        new_energy = math.floor(self._current_energy / REPRODUCE_NEW_COEF)
        new_population_size = math.floor(self._population_size / REPRODUCE_POPULATION_COEF)

        if new_energy <= 0 or new_population_size <= 0:
            print(f"Организм {self._id} не имеет достаточно ресурсов для размножения (E: {self._current_energy}, P: {self._population_size}).")
            return None

        child = type(self)()
        child._current_age = 0
        child._current_energy = new_energy
        child._population_size = new_population_size

        self._current_energy = math.floor(self._current_energy / REPRODUCE_OLD_COEF)

        print(f"Организм {self._id} (популяция: {self._population_size}) размножился. "
              f"Появился новый организм {child._id} (популяция: {child._population_size}). "
              f"Энергия родителя уменьшена до {self._current_energy}, энергия потомка: {child._current_energy}.")
        return child

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def __repr__(self):
        pass

    @abstractmethod
    def replenish_energy(self):
        pass

# 2. Concrete implementations (2 балла)

Реализуй два класса - Herbivore (травоядные) и Plant (растения). Оба должны наследоваться от Organism.

Для Herbivore:

- Инициализатор, который проставляет `max_age` равным `MAX_HERBIVORE_AGE`
- `replenish_energy(eaten_plant_energy)`, который увеличивает количество энергии на `eaten_plant_energy`.


Для Plant:

- Инициализатор, который проставляет `max_age` равным `MAX_PLANT_AGE`
- `replenish_energy` увеличивает текущий запас энергии на `PLANT_ENERGY_INC_COEF`

In [ ]:
MAX_HERBIVORE_AGE = 50
MAX_PLANT_AGE = 20
PLANT_ENERGY_INC_COEF = 5

class Herbivore(Organism):
    def __init__(self):
        super().__init__()
        self._max_age = MAX_HERBIVORE_AGE

    def __str__(self):
        return (f"Herbivore ID: {self._id}, Age: {self._current_age}, "
                f"Energy: {self._current_energy}, Population: {self._population_size}, "
                f"Alive: {self.alive}")

    def __repr__(self):
        return self.__str__()

    def replenish_energy(self, eaten_plant_energy: int):
        self._current_energy += eaten_plant_energy
        print(f"Herbivore {self._id} пополнил энергию на {eaten_plant_energy}. Текущая энергия: {self._current_energy}")


class Plant(Organism):
    def __init__(self):
        super().__init__()
        self._max_age = MAX_PLANT_AGE

    def __str__(self):
        return (f"Plant ID: {self._id}, Age: {self._current_age}, "
                f"Energy: {self._current_energy}, Population: {self._population_size}, "
                f"Alive: {self.alive}")

    def __repr__(self):
        return self.__str__()

    def replenish_energy(self):
        self._current_energy += PLANT_ENERGY_INC_COEF
        print(f"Plant {self._id} пополнило энергию на {PLANT_ENERGY_INC_COEF}. Текущая энергия: {self._current_energy}")

Напиши несколько примеров работы с этими классами - протестируй таким образом, что всё ок

In [ ]:
# Сбросим счетчик ID для чистоты тестового примера
ID_COUNTER = 0

print("\n--- Тестирование класса Herbivore ---")
# Создаем травоядное
herb_1 = Herbivore()
herb_1._population_size = 10
herb_1._current_energy = 15
print(f"Создан: {herb_1}")
print(f"Herbivore ID {herb_1._id} жив? {bool(herb_1)}")

# Пополняем энергию травоядного
herb_1.replenish_energy(5)
print(f"После пополнения энергии: {herb_1}")

# Травоядное размножается
herb_child = herb_1.reproduce()
if herb_child:
    print(f"Родитель после размножения: {herb_1}")
    print(f"Потомок травоядного: {herb_child}")

# Убиваем травоядное через сеттер alive
print(f"\nУбиваем Herbivore ID {herb_1._id}...")
herb_1.alive = False
print(f"Herbivore ID {herb_1._id} жив? {bool(herb_1)}")
print(f"Состояние Herbivore ID {herb_1._id} после смерти: Age={herb_1._current_age}, Energy={herb_1._current_energy}")

# Пытаемся оживить травоядное
print(f"\nПытаемся оживить Herbivore ID {herb_1._id}...")
herb_1.alive = True
print(f"Herbivore ID {herb_1._id} жив? {bool(herb_1)}")
print(f"Состояние Herbivore ID {herb_1._id} после оживления: Age={herb_1._current_age}, Energy={herb_1._current_energy}")



print("\n--- Тестирование класса Plant ---")
# Создаем растение
plant_1 = Plant()
plant_1._population_size = 20
plant_1._current_energy = 10
print(f"Создано: {plant_1}")
print(f"Plant ID {plant_1._id} живо? {bool(plant_1)}")

# Пополняем энергию растения
plant_1.replenish_energy()
print(f"После пополнения энергии: {plant_1}")

# Растение размножается
plant_child = plant_1.reproduce()
if plant_child:
    print(f"Родитель после размножения: {plant_1}")
    print(f"Потомок растения: {plant_child}")

# Проверяем условия смерти (например, возраст)
print(f"\nПроверяем смерть растения по возрасту...")
plant_2 = Plant()
plant_2._population_size = 50
plant_2._current_age = MAX_PLANT_AGE
plant_2._current_energy = 5
print(f"Создано: {plant_2}")
print(f"Plant ID {plant_2._id} живо? {bool(plant_2)}")
# Увеличиваем возраст, чтобы растение умерло
plant_2._current_age += 1
print(f"Plant ID {plant_2._id} живо после увеличения возраста? {bool(plant_2)}")

print("\n--- Тестирование воспроизведения без достаточной энергии/популяции ---")
herb_low_res = Herbivore()
herb_low_res._population_size = 1
herb_low_res._current_energy = 1
print(f"\nСоздан травоядный с низкими ресурсами: {herb_low_res}")
herb_low_res_child = herb_low_res.reproduce()
if not herb_low_res_child:
    print(f"Организм {herb_low_res._id} не смог размножиться, как и ожидалось.")


--- Тестирование класса Herbivore ---
Создан: Herbivore ID: 0, Age: 0, Energy: 15, Population: 10, Alive: True
Herbivore ID 0 жив? True
Herbivore 0 пополнил энергию на 5. Текущая энергия: 20
После пополнения энергии: Herbivore ID: 0, Age: 0, Energy: 20, Population: 10, Alive: True
Организм 0 (популяция: 10) размножился. Появился новый организм 1 (популяция: 5). Энергия родителя уменьшена до 10, энергия потомка: 10.
Родитель после размножения: Herbivore ID: 0, Age: 0, Energy: 10, Population: 10, Alive: True
Потомок травоядного: Herbivore ID: 1, Age: 0, Energy: 10, Population: 5, Alive: True

Убиваем Herbivore ID 0...
Herbivore ID 0 жив? False
Состояние Herbivore ID 0 после смерти: Age=51, Energy=10

Пытаемся оживить Herbivore ID 0...
Herbivore ID 0 жив? True
Состояние Herbivore ID 0 после оживления: Age=49, Energy=1

--- Тестирование класса Plant ---
Создано: Plant ID: 2, Age: 0, Energy: 10, Population: 20, Alive: True
Plant ID 2 живо? True
Plant 2 пополнило энергию на 5. Текущая энерг

В каком порядке должен вызываться (и вызываются у тебя):
- метод `spawn`
- `__init__` у ребёнка (`Plant/Herbivore`)
- `__init__` у родителя (`Organism`)?

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

сначала spawn, потом init у ребенка, потом init у родителя, у меня вызывается в том же порядке

# 3. Нюансы работы с классами (3 балла)


В следующих ячейках изучи вопросы копирования классов как объектов (НЕ экземпляров! Именно самих классов) и последующего их сравнения. Также изучи, можно ли поменять `__bases__` и/или `__mro__` после создания класса как объекта. Если что-то сходу не получается - погугли :)

In [4]:
import copy

print("--- 1. Копирование классов ---")
class MyClass:
    pass

print(f"Оригинальный класс MyClass: {MyClass}, id: {id(MyClass)}")

# 1. Простое присваивание (не копирование, а ссылка)
MyClassAlias = MyClass
print(f"MyClassAlias (присваивание): {MyClassAlias}, id: {id(MyClassAlias)}")
print(f"MyClass is MyClassAlias: {MyClass is MyClassAlias}") # True

# 2. Попытка поверхностного копирования (copy.copy)
try:
    # copy.copy для классов обычно просто возвращает сам класс, это не создает новый класс
    MyClassShallowCopy = copy.copy(MyClass)
    print(f"MyClassShallowCopy (copy.copy): {MyClassShallowCopy}, id: {id(MyClassShallowCopy)}")
    print(f"MyClass is MyClassShallowCopy: {MyClass is MyClassShallowCopy}") # True
except TypeError as e:
    print(f"Ошибка при copy.copy(MyClass): {e}")

# 3. Попытка глубокого копирования (copy.deepcopy)
try:
    # copy.deepcopy для классов обычно не работает или вызывает ошибку
    # или возвращает тот же класс, если нет специальных методов __reduce__/__deepcopy__
    MyClassDeepCopy = copy.deepcopy(MyClass)
    print(f"MyClassDeepCopy (copy.deepcopy): {MyClassDeepCopy}, id: {id(MyClassDeepCopy)}")
    print(f"MyClass is MyClassDeepCopy: {MyClass is MyClassDeepCopy}") # True
except TypeError as e:
    print(f"Ошибка при copy.deepcopy(MyClass): {e}")
except Exception as e:
    print(f"Другая ошибка при copy.deepcopy(MyClass): {e}")

# Создание нового класса с помощью type()
# Это способ создания нового класса, который 'копирует' структуру другого
NewClassFromType = type('NewClassFromType', MyClass.__bases__, dict(MyClass.__dict__))
print(f"NewClassFromType (через type()): {NewClassFromType}, id: {id(NewClassFromType)}")
print(f"MyClass is NewClassFromType: {MyClass is NewClassFromType}") # False

print("\n--- 2. Сравнение классов ---")
class AnotherClass:
    pass

class SameNameClass:
    pass

SameNameClassAlias = SameNameClass

print(f"MyClass == MyClassAlias: {MyClass == MyClassAlias}") # True
print(f"MyClass is MyClassAlias: {MyClass is MyClassAlias}") # True

print(f"MyClass == AnotherClass: {MyClass == AnotherClass}") # False
print(f"MyClass is AnotherClass: {MyClass is AnotherClass}") # False

print(f"SameNameClass == SameNameClassAlias: {SameNameClass == SameNameClassAlias}") # True
print(f"SameNameClass is SameNameClassAlias: {SameNameClass is SameNameClassAlias}") # True

print(f"MyClass == NewClassFromType: {MyClass == NewClassFromType}") # False
print(f"MyClass is NewClassFromType: {MyClass is NewClassFromType}") # False

print("\n--- 3. Изменение __bases__ и __mro__ после создания класса ---")
class BaseA:
    pass

class BaseB:
    pass

class MyChild(BaseA):
    def get_parent_name(self):
        return "BaseA method"

print(f"Исходные базы MyChild: {MyChild.__bases__}")
print(f"Исходный MRO MyChild: {MyChild.__mro__}")

# 3.1. Изменение __bases__ (может быть успешно)
try:
    print("\nПопытка изменить MyChild.__bases__ на (BaseB,)")
    MyChild.__bases__ = (BaseB, )
    print(f"Новые базы MyChild: {MyChild.__bases__}")
    print(f"Новый MRO MyChild: {MyChild.__mro__}") # MRO обновляется автоматически
    # Проверим, как это повлияет на новые экземпляры
    new_child_instance = MyChild()
    # Если BaseB не имеет get_parent_name, будет AttributeError, если не переопределено
    # print(new_child_instance.get_parent_name()) # Это вызовет ошибку, если BaseB не имеет такого метода
except TypeError as e:
    print(f"Ошибка при изменении __bases__: {e}")

# 3.2. Изменение __bases__ с ошибкой (например, из-за internal deallocator)
# Эта ошибка 'deallocator differs' часто связана с внутренними механизмами CPython
# или специфическими метаклассами. Воспроизвести ее простым классом сложно.
# Но вот пример, который демонстрирует, как Python не дает менять bases,
# если это ломает внутреннюю консистентность.
class Foo:
    pass

class Bar(object):
    pass

try:
    print("\nПопытка вызвать 'deallocator differs' TypeError:")
    # Это может не воспроизвести ту же ошибку, что у пользователя,
    # но демонстрирует, что не любое изменение __bases__ допустимо.
    Bar.__bases__ = (Foo,)
    print(f"Bar bases changed to: {Bar.__bases__}")
except TypeError as e:
    print(f"Ошибка при изменении __bases__ (например, 'deallocator differs'): {e}")

# 3.3. Изменение __bases__ с нарушением MRO (Method Resolution Order)
class GParent:
    pass
class P1(GParent):
    pass
class P2(GParent):
    pass

class ChildMRO(P1, P2):
    pass

print(f"\nИсходные MRO ChildMRO: {ChildMRO.__mro__}")

try:
    # Пример создания класса с неразрешимым MRO
    class X(object):
        pass
    class Y(object):
        pass
    class A_MRO(X, Y):
        pass
    class B_MRO(Y, X):
        pass
    print("Попытка создать класс с неразрешимым MRO (C_MRO(A_MRO, B_MRO))")
    class C_MRO(A_MRO, B_MRO):
        pass
except TypeError as e:
    print(f"Ошибка MRO при создании класса (TypeError): {e}")

# 3.4. Попытка изменить __mro__ напрямую (всегда ошибка)
try:
    print("\nПопытка изменить MyChild.__mro__ напрямую:")
    # __mro__ является read-only атрибутом
    MyChild.__mro__ = (MyChild, object) # Это вызовет AttributeError
    print(f"Измененный MRO MyChild: {MyChild.__mro__}")
except AttributeError as e:
    print(f"Ошибка при изменении __mro__ напрямую (AttributeError): {e}")
except Exception as e:
    print(f"Другая ошибка при изменении __mro__ напрямую: {e}")

# 3.5. Влияние изменения __bases__ на существующие и новые экземпляры
class DynamicParent:
    def get_info(self):
        return "Info from original parent"

class DynamicChild(DynamicParent):
    pass

print("\n--- Влияние изменения __bases__ на экземпляры ---")
instance1 = DynamicChild()
print(f"Экземпляр до изменения bases: {instance1.get_info()}")

class NewDynamicParent:
    def get_info(self):
        return "Info from new parent"

# Изменяем базы класса
DynamicChild.__bases__ = (NewDynamicParent,)
print(f"Новые базы DynamicChild: {DynamicChild.__bases__}")

# Существующий экземпляр не изменится
print(f"Существующий экземпляр (instance1) после изменения bases: {instance1.get_info()}")

# Новый экземпляр будет использовать новую иерархию
instance2 = DynamicChild()
print(f"Новый экземпляр (instance2) после изменения bases: {instance2.get_info()}")

# Можно изменить класс существующего экземпляра напрямую
print("\n--- Изменение __class__ существующего экземпляра ---")
print(f"instance1.__class__ до изменения: {instance1.__class__}")
instance1.__class__ = NewDynamicParent # Изменяем класс существующего объекта
print(f"instance1.__class__ после изменения: {instance1.__class__}")
print(f"Экземпляр instance1 после изменения __class__: {instance1.get_info()}")

--- 1. Копирование классов ---
Оригинальный класс MyClass: <class '__main__.MyClass'>, id: 397977472
MyClassAlias (присваивание): <class '__main__.MyClass'>, id: 397977472
MyClass is MyClassAlias: True
MyClassShallowCopy (copy.copy): <class '__main__.MyClass'>, id: 397977472
MyClass is MyClassShallowCopy: True
MyClassDeepCopy (copy.deepcopy): <class '__main__.MyClass'>, id: 397977472
MyClass is MyClassDeepCopy: True
NewClassFromType (через type()): <class '__main__.NewClassFromType'>, id: 398594896
MyClass is NewClassFromType: False

--- 2. Сравнение классов ---
MyClass == MyClassAlias: True
MyClass is MyClassAlias: True
MyClass == AnotherClass: False
MyClass is AnotherClass: False
SameNameClass == SameNameClassAlias: True
SameNameClass is SameNameClassAlias: True
MyClass == NewClassFromType: False
MyClass is NewClassFromType: False

--- 3. Изменение __bases__ и __mro__ после создания класса ---
Исходные базы MyChild: (<class '__main__.BaseA'>,)
Исходный MRO MyChild: (<class '__main__.

Какие выводы ты можешь сделать? Какие есть нюансы:

1. Копирования классов?

2. Сравнения классов?

3. Изменения `__bases__` и `__mro__` после создания класса?

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

Нельзя создать новый класс в памяти через copy.copy(class), также нельзя через copy.deepcopy(class) потому что class это иммутабельный объект, можно копировать через type("NewClass", MyClass.__bases__, dict(MyClass.__dict__)), где bases это кортеж с родительскими классами, его можно менять, mro это определенный порядок, в котором Python ищет атрибут или метод в иерархии классов, его менять нельзя


:Теперь изучи следующие два примера и проанализируй.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [4]:
class A:
    pass

class B:
    pass

B.__bases__ = (A, )


TypeError: __bases__ assignment: 'A' deallocator differs from 'object'

In [3]:
class A:
    pass

class B:
    pass

class C(A):
    pass

C.__bases__ = (B, )


Выводы:

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

В первом случае ошибка, потому что B это класс, который наследуется от object, нельзя заменить object. Во втором случае нет никаких противоречий, потому что мы просто меняем класс, от которого наследуется С.

Теперь попробуй посоздавать экземпляры классов и поменять у них классы динамически. Всегда ли это получается? Почему?

In [5]:
class A:
    def say_hello(self):
        return "Привет от класса А!"

class B:
    def say_bye(self):
        return "Пока от класса B!"

# 1. Создаем экземпляр A
a = A()
print(f"До мутации: {a.say_hello()} (Тип: {type(a)})")

# 2. Меняем класс динамически
a.__class__ = B
print(f"После мутации: {a.say_bye()} (Тип: {type(a)})")

# Попытка вызвать старый метод вызовет ошибку, так как объект 'a' теперь официально B
# print(a.say_hello()) # AttributeError

# 3. Пытаемся превратить в int (спойлер: это упадет)
try:
    a.__class__ = int
except TypeError as e:
    print(f"Ошибка при смене на int: {e}")

До мутации: Привет от класса А! (Тип: <class '__main__.A'>)
После мутации: Пока от класса B! (Тип: <class '__main__.B'>)
Ошибка при смене на int: __class__ assignment only supported for mutable types or ModuleType subclasses


Выводы:

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

Нельзя превратить обычный объект в системную штуку типо int, list ..., но можно заменить на другой

# 4. Mixin'ы. (5 баллов)


Наша цель - в Runtime добавлять к классам новые характеристики, логика которых будет описана в Mixin'ах.

Подумай, как надо поменять реализацию классов `Herbivore` и `Plant`, чтобы легко вызывались методы из Mixin'ов, но при этом если их нет, тоже ничего не ломалось.

Как это сделать?

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>

Для начала определим все необходимые константы для Mixin'ов и общей симуляции. Мы также переопределим некоторые существующие константы, чтобы они были в одном месте для удобства управления.

In [22]:
from abc import ABC, abstractmethod
import math
import random

ID_COUNTER = 0
REPRODUCE_NEW_COEF = 2
REPRODUCE_POPULATION_COEF = 2
REPRODUCE_OLD_COEF = 2

MAX_HERBIVORE_AGE = 50
MAX_PLANT_AGE = 20

PLANT_ENERGY_INC_COEF = 5

SHORT_LIFESPAN_COEF = 0.5
SHORT_LIFESPAN_ENERGY_BONUS = 5

LONG_LIFESPAN_COEF = 2
LONG_LIFESPAN_ENERGY_PENALTY = 2

HIGH_FERTILITY_CHILD_POP_MULT = 2
HIGH_FERTILITY_OLD_ENERGY_COST_MULT = 2

TOXIC_DAMAGE_COEF = 0.5

RESILIENT_TOXIC_REDUCTION_COEF = 0.5

MUTATION_POSSIBILITY = 0.2

HERBIVORE_ENERGY_CONSUMPTION = 1
PLANT_ENERGY_CONSUMPTION = 1

PLANT_EATEN_POP_COEF = 0.1
PLANT_EATEN_ENERGY_COEF = 0.2

Теперь мы переопределим абстрактный класс `Organism` из пункта 1, добавив в него метод `mutate`. Этот метод будет отвечать за динамическое изменение списка родительских классов (Mixin'ов) у дочерних организмов.

In [23]:
class Organism(ABC):
    _possible_mixins = []

    def __init__(self):
        global ID_COUNTER
        self._population_size = 0
        self._current_age = 0
        self._current_energy = 1
        self._id = ID_COUNTER
        ID_COUNTER += 1
        self._max_age = None
        self._energy_replenish_rate = 0

    @property
    def alive(self):
        if self._max_age is None:
             return self._current_energy > 0
        return self._current_age < self._max_age and self._current_energy > 0

    @alive.setter
    def alive(self, value):
        if bool(self.alive) == bool(value):
            return

        if value and not self.alive:
            self._current_age = self._max_age - 1 if self._max_age is not None else 0
            self._current_energy = 1
        elif not value and self.alive:
            self._current_age = self._max_age + 1 if self._max_age is not None else 1

    def __bool__(self):
        return self.alive

    @classmethod
    def spawn(cls, population_size, current_age):
        obj = cls()
        obj._population_size = population_size
        obj._current_age = current_age
        return obj

    def reproduce(self):
        if not self.alive:
            print(f"Организм {self._id} мертв и не может размножаться.")
            return None

        new_energy = math.floor(self._current_energy / REPRODUCE_NEW_COEF)
        new_population_size = math.floor(self._population_size / REPRODUCE_POPULATION_COEF)

        if new_energy <= 0 or new_population_size <= 0:
            print(f"Организм {self._id} не имеет достаточно ресурсов для размножения (E: {self._current_energy}, P: {self._population_size}).")
            return None

        child = type(self)()
        child._current_age = 0
        child._current_energy = new_energy
        child._population_size = new_population_size

        self._current_energy = math.floor(self._current_energy / REPRODUCE_OLD_COEF)

        print(f"Организм {self._id} (популяция: {self._population_size}) размножился. "
              f"Появился новый организм {child._id} (популяция: {child._population_size}). "
              f"Энергия родителя уменьшена до {self._current_energy}, энергия потомка: {child._current_energy}.")
        return child

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def __repr__(self):
        pass

    @abstractmethod
    def replenish_energy(self):
        pass

    def mutate(self):
        original_concrete_class = None
        for base_cls in self.__class__.__mro__:
            if base_cls is Herbivore:
                original_concrete_class = Herbivore
                break
            if base_cls is Plant:
                original_concrete_class = Plant
                break
        if original_concrete_class is None:
            print(f"Ошибка мутации: Не удалось определить базовый класс для организма {self._id}. Текущий MRO: {self.__class__.__mro__}")
            return

        applicable_mixins_pool = original_concrete_class._possible_mixins

        current_applied_mixins = [b for b in self.__class__.__mro__ if b in applicable_mixins_pool and b is not original_concrete_class]

        if random.random() < MUTATION_POSSIBILITY:
            available_mixins_to_add = [m for m in applicable_mixins_pool if m not in current_applied_mixins]

            if available_mixins_to_add:
                new_mixin = random.choice(available_mixins_to_add)

                new_bases_tuple = tuple([new_mixin] + current_applied_mixins + [original_concrete_class])
                new_bases_tuple = tuple(dict.fromkeys(new_bases_tuple))

                base_name = original_concrete_class.__name__
                mixin_names = [m.__name__ for m in new_bases_tuple if m in applicable_mixins_pool]
                new_class_name = "_".join([base_name] + sorted(mixin_names)) if mixin_names else base_name

                new_type = type(new_class_name, new_bases_tuple, dict(self.__class__.__dict__))

                self.__class__ = new_type
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) мутировал: добавлен Mixin '{new_mixin.__name__}'.")

                if hasattr(new_mixin, '__init__') and new_mixin.__init__ != object.__init__ and new_mixin.__init__ != Organism.__init__:
                    try:
                        new_mixin.__init__(self)
                    except Exception as e:
                        print(f"Предупреждение: Не удалось применить __init__ Mixin'а {new_mixin.__name__} к организму {self._id} при мутации: {e}")
            else:
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) не смог мутировать: нет доступных Mixin'ов для добавления.")
        else:
            if current_applied_mixins:
                mixin_to_remove = random.choice(current_applied_mixins)

                remaining_mixins = [m for m in current_applied_mixins if m != mixin_to_remove]

                new_bases_tuple = tuple(remaining_mixins + [original_concrete_class])
                new_bases_tuple = tuple(dict.fromkeys(new_bases_tuple))

                base_name = original_concrete_class.__name__
                mixin_names = [m.__name__ for m in new_bases_tuple if m in applicable_mixins_pool]
                new_class_name = "_".join([base_name] + sorted(mixin_names)) if mixin_names else base_name

                new_type = type(new_class_name, new_bases_tuple, dict(self.__class__.__dict__))
                self.__class__ = new_type
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) мутировал: удален Mixin '{mixin_to_remove.__name__}'.")
            else:
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) не смог мутировать: нет Mixin'ов для удаления.")

Скопируй `Herbivore` и `Plant` и добавь изменения в них, чтобы работало корректно в будущем

In [24]:
MAX_HERBIVORE_AGE = 50
MAX_PLANT_AGE = 20
PLANT_ENERGY_INC_COEF = 5

class Herbivore(Organism):
    _possible_mixins = [ShortLifespan, LongLifespan, HighFertilityMixin, IdentifiesToxicPlants, ResilientToToxicPlants]

    def __init__(self):
        super().__init__()
        self._max_age = MAX_HERBIVORE_AGE

    def __str__(self):
        return (f"Herbivore ID: {self._id}, Age: {self._current_age}, "
                f"Energy: {self._current_energy}, Population: {self._population_size}, "
                f"Alive: {self.alive}")

    def __repr__(self):
        return self.__str__()

    def replenish_energy(self, eaten_plant_energy: int):
        self._current_energy += eaten_plant_energy
        print(f"Herbivore {self._id} пополнил энергию на {eaten_plant_energy}. Текущая энергия: {self._current_energy}")


class Plant(Organism):
    _possible_mixins = [ShortLifespan, LongLifespan, HighFertilityMixin, Toxic]

    def __init__(self):
        super().__init__()
        self._max_age = MAX_PLANT_AGE

    def __str__(self):
        return (f"Plant ID: {self._id}, Age: {self._current_age}, "
                f"Energy: {self._current_energy}, Population: {self._population_size}, "
                f"Alive: {self.alive}")

    def __repr__(self):
        return self.__str__()

    def replenish_energy(self):
        self._current_energy += PLANT_ENERGY_INC_COEF
        print(f"Plant {self._id} пополнило энергию на {PLANT_ENERGY_INC_COEF}. Текущая энергия: {self._current_energy}")

Напиши следующие Mixin'ы:

- `ShortLifespan` - уменьшает максимальный возраст организма в `SHORT_LIFESPAN_COEF` раз, но увеличивает прибавку энергии в `SHORT_LIFESPAN_ENERGY_COEF` раз

- `LongLifespan` - обратное предыдущему. Добавь соответствующие константы.

- `HighFertilityMixin` - увеличивает у возвращаемого "ребёнка" в методе `reproduce` `population_size` в `HIGH_FERTILITY_COEF` раз, но повышает затраты энергии в `reproduce` в `HIGH_FERTILITY_ENERGY_COEF` раз.

- `Toxic` (только для растений) - при поедании такого растения количество животных из данного набора уменьшается в `TOXIC_COEF` раз.

- `IdentifiesToxicPlants` (только для травоядных) - при выборе растения в фазе поедания, если растение токсично, выбирают не есть его.

- `ResilientToToxicPlants` (только для травоядных) - при выборе токсичного растения смертность уменьшена в `RESILIENT_COEF` раз.

In [25]:
class ShortLifespan:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if self._max_age is not None:
            self._max_age = math.floor(self._max_age * SHORT_LIFESPAN_COEF)
        print(f"Mixin: {self.__class__.__name__} активирован для ID {self._id}. Новый _max_age: {self._max_age}")

    def replenish_energy(self, *args, **kwargs):
        super().replenish_energy(*args, **kwargs)
        self._current_energy += SHORT_LIFESPAN_ENERGY_BONUS
        print(f"Mixin: {self.__class__.__name__} (ID {self._id}) получил бонус энергии {SHORT_LIFESPAN_ENERGY_BONUS}. Текущая энергия: {self._current_energy}")

class LongLifespan:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if self._max_age is not None:
            self._max_age = math.floor(self._max_age * LONG_LIFESPAN_COEF)
        print(f"Mixin: {self.__class__.__name__} активирован для ID {self._id}. Новый _max_age: {self._max_age}")

    def replenish_energy(self, *args, **kwargs):
        super().replenish_energy(*args, **kwargs)
        self._current_energy = math.floor(self._current_energy / LONG_LIFESPAN_ENERGY_PENALTY)
        print(f"Mixin: {self.__class__.__name__} (ID {self._id}) получил штраф энергии. Текущая энергия: {self._current_energy}")

class HighFertilityMixin:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print(f"Mixin: {self.__class__.__name__} активирован для ID {self._id}")

    def reproduce(self):
        if not self.alive:
            print(f"Организм {self._id} мертв и не может размножаться.")
            return None

        new_energy_base = math.floor(self._current_energy / REPRODUCE_NEW_COEF)
        new_population_size_base = math.floor(self._population_size / REPRODUCE_POPULATION_COEF)

        new_population_size = new_population_size_base * HIGH_FERTILITY_CHILD_POP_MULT
        parent_energy_cost = REPRODUCE_OLD_COEF * HIGH_FERTILITY_OLD_ENERGY_COST_MULT

        if new_energy_base <= 0 or new_population_size <= 0:
            print(f"Организм {self._id} не имеет достаточно ресурсов для размножения (E: {self._current_energy}, P: {self._population_size}).")
            return None

        child = type(self)()
        child._current_age = 0
        child._current_energy = new_energy_base
        child._population_size = new_population_size

        self._current_energy = math.floor(self._current_energy / parent_energy_cost)

        print(f"Mixin: {self.__class__.__name__} (ID {self._id}) размножился с повышенной плодовитостью. "
              f"Появился новый организм {child._id} (популяция: {child._population_size}). "
              f"Энергия родителя уменьшена до {self._current_energy}, энергия потомка: {child._current_energy}.")
        return child

class Toxic:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.is_toxic = True
        print(f"Mixin: {self.__class__.__name__} активирован для ID {self._id}. Растение токсично.")

class IdentifiesToxicPlants:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print(f"Mixin: {self.__class__.__name__} активирован для ID {self._id}.")

    def _should_eat(self, plant):
        if hasattr(plant, 'is_toxic') and plant.is_toxic:
            print(f"Mixin: {self.__class__.__name__} (ID {self._id}) обнаружил токсичное растение (ID {plant._id}) и решил его не есть.")
            return False
        return True

class ResilientToToxicPlants:
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print(f"Mixin: {self.__class__.__name__} активирован для ID {self._id}.")

    def _apply_toxic_effect(self, plant, population_reduction):
        if hasattr(plant, 'is_toxic') and plant.is_toxic:
            reduced_population_reduction = math.floor(population_reduction * RESILIENT_TOXIC_REDUCTION_COEF)
            print(f"Mixin: {self.__class__.__name__} (ID {self._id}) устойчив к токсичности. Урон снижен с {population_reduction} до {reduced_population_reduction}.")
            return reduced_population_reduction
        return population_reduction

Перекопируй код для абстрактного класса из п. 1 данной работы и добавь к нему метод `mutate`: с вероятностью `MUTATION_POSSIBILITY` процентов он выбирает какой-то из mixin'ов, который у него ещё не стоит в родителях, и добавляет его в родителей; иначе - он выбирает одного из родителей и удаляет его из кортежа родителей (`__bases__`).


Подумай, как это реализовать так, чтобы сам метод был общий и для `Plant`, и для `Herbivore`, но при этом выбор возможных Mixin'ов был разный. При надобности их тоже скопируй ниже и поправь в них код.


Также важно! в методе `mutate` должно быть логгирование (`print` с описанием произошедшего), чтобы было понятно, кто мутировал (и во что).

In [36]:
class Organism(ABC):
    _possible_mixins = []
    _dynamic_class_cache = {}

    def __init__(self):
        global ID_COUNTER
        self._population_size = 0
        self._current_age = 0
        self._current_energy = 1
        self._id = ID_COUNTER
        ID_COUNTER += 1
        self._max_age = None
        self._energy_replenish_rate = 0

    @property
    def alive(self):
        if self._max_age is None:
             return self._current_energy > 0
        return self._current_age < self._max_age and self._current_energy > 0

    @alive.setter
    def alive(self, value):
        if bool(self.alive) == bool(value):
            return

        if value and not self.alive:
            self._current_age = self._max_age - 1 if self._max_age is not None else 0
            self._current_energy = 1
        elif not value and self.alive:
            self._current_age = self._max_age + 1 if self._max_age is not None else 1

    def __bool__(self):
        return self.alive

    @classmethod
    def spawn(cls, population_size, current_age):
        obj = cls()
        obj._population_size = population_size
        obj._current_age = current_age
        return obj

    def reproduce(self):
        if not self.alive:
            print(f"Организм {self._id} мертв и не может размножаться.")
            return None

        new_energy = math.floor(self._current_energy / REPRODUCE_NEW_COEF)
        new_population_size = math.floor(self._population_size / REPRODUCE_POPULATION_COEF)

        if new_energy <= 0 or new_population_size <= 0:
            print(f"Организм {self._id} не имеет достаточно ресурсов для размножения (E: {self._current_energy}, P: {self._population_size}).")
            return None

        child = type(self)()
        child._current_age = 0
        child._current_energy = new_energy
        child._population_size = new_population_size

        self._current_energy = math.floor(self._current_energy / REPRODUCE_OLD_COEF)

        print(f"Организм {self._id} (популяция: {self._population_size}) размножился. "
              f"Появился новый организм {child._id} (популяция: {child._population_size}). "
              f"Энергия родителя уменьшена до {self._current_energy}, энергия потомка: {child._current_energy}.")
        return child

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def __repr__(self):
        pass

    @abstractmethod
    def replenish_energy(self):
        pass

    def mutate(self):
        original_concrete_class = None
        for base_cls in self.__class__.__mro__:
            if base_cls is Herbivore:
                original_concrete_class = Herbivore
                break
            if base_cls is Plant:
                original_concrete_class = Plant
                break
        if original_concrete_class is None:
            print(f"Ошибка мутации: Не удалось определить базовый класс для организма {self._id}. Текущий MRO: {self.__class__.__mro__}")
            return

        applicable_mixins_pool = original_concrete_class._possible_mixins

        current_applied_mixins = [b for b in self.__class__.__mro__ if b in applicable_mixins_pool and b is not original_concrete_class]

        if random.random() < MUTATION_POSSIBILITY:
            available_mixins_to_add = [m for m in applicable_mixins_pool if m not in current_applied_mixins]

            if available_mixins_to_add:
                new_mixin = random.choice(available_mixins_to_add)

                new_bases_tuple = tuple([new_mixin] + current_applied_mixins + [original_concrete_class])
                new_bases_tuple = tuple(dict.fromkeys(new_bases_tuple))

                base_name = original_concrete_class.__name__
                mixin_names = [m.__name__ for m in new_bases_tuple if m in applicable_mixins_pool]
                new_class_name = "_".join([base_name] + sorted(mixin_names)) if mixin_names else base_name

                class_key = (new_class_name, new_bases_tuple) # Use for caching
                if class_key in Organism._dynamic_class_cache:
                    new_type = Organism._dynamic_class_cache[class_key]
                else:
                    new_type = type(new_class_name, new_bases_tuple, {})
                    Organism._dynamic_class_cache[class_key] = new_type

                self.__class__ = new_type
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) мутировал: добавлен Mixin '{new_mixin.__name__}'.")

                if hasattr(new_mixin, '__init__') and new_mixin.__init__ != object.__init__ and new_mixin.__init__ != Organism.__init__:
                    try:

                        new_mixin.__init__(self)
                    except Exception as e:
                        print(f"Предупреждение: Не удалось применить __init__ Mixin'а {new_mixin.__name__} к организму {self._id} при мутации: {e}")
            else:
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) не смог мутировать: нет доступных Mixin'ов для добавления.")
        else:
            if current_applied_mixins:
                mixin_to_remove = random.choice(current_applied_mixins)

                remaining_mixins = [m for m in current_applied_mixins if m != mixin_to_remove]

                new_bases_tuple = tuple(remaining_mixins + [original_concrete_class])
                new_bases_tuple = tuple(dict.fromkeys(new_bases_tuple))

                base_name = original_concrete_class.__name__
                mixin_names = [m.__name__ for m in new_bases_tuple if m in applicable_mixins_pool]
                new_class_name = "_".join([base_name] + sorted(mixin_names)) if mixin_names else base_name

                class_key = (new_class_name, new_bases_tuple) # Use for caching
                if class_key in Organism._dynamic_class_cache:
                    new_type = Organism._dynamic_class_cache[class_key]
                else:
                    new_type = type(new_class_name, new_bases_tuple, {})
                    Organism._dynamic_class_cache[class_key] = new_type

                self.__class__ = new_type
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) мутировал: удален Mixin '{mixin_to_remove.__name__}'.")
            else:
                print(f"Организм {self._id} (Тип: {self.__class__.__name__}) не смог мутировать: нет Mixin'ов для удаления.")

Напиши пару примеров работы с `mutate`, чтобы протестировать, что всё сделано хорошо.

In [37]:
ID_COUNTER = 0

print("--- Тестирование метода mutate для Herbivore ---")
herb = Herbivore()
herb._population_size = 10
herb._current_energy = 10
print(f"Начальное состояние: {herb} (Max Age: {herb._max_age})")

for i in range(5):
    print(f"\n--- Мутация Herbivore #{i+1} ---")
    herb.mutate()
    print(f"Текущее состояние: {herb} (Max Age: {herb._max_age})")

print("\n--- Тестирование метода mutate для Plant ---")
plant = Plant()
plant._population_size = 20
plant._current_energy = 15
print(f"Начальное состояние: {plant} (Max Age: {plant._max_age})")

for i in range(5):
    print(f"\n--- Мутация Plant #{i+1} ---")
    plant.mutate()
    print(f"Текущее состояние: {plant} (Max Age: {plant._max_age})")

print("\n--- Тестирование влияния мутаций на Max Age ---")
herb_age_test = Herbivore()
print(f"Начальное Herbivore (ID {herb_age_test._id}): Max Age = {herb_age_test._max_age}")

if ShortLifespan not in herb_age_test.__class__._possible_mixins:
    herb_age_test.__class__._possible_mixins.append(ShortLifespan)

if LongLifespan not in herb_age_test.__class__._possible_mixins:
    herb_age_test.__class__._possible_mixins.append(LongLifespan)

if ShortLifespan not in [b for b in herb_age_test.__class__.__mro__ if b in herb_age_test.__class__._possible_mixins]:
    current_bases = list(herb_age_test.__class__.__mro__)
    original_concrete = Herbivore
    new_bases = tuple([ShortLifespan] + [b for b in current_bases if b not in [Organism, ABC, object, ShortLifespan]] )
    new_type = type("Herbivore_ShortLifespan", new_bases, dict(herb_age_test.__class__.__dict__))
    herb_age_test.__class__ = new_type
    ShortLifespan.__init__(herb_age_test)
    print(f"После добавления ShortLifespan: {herb_age_test.__class__.__name__}, Max Age = {herb_age_test._max_age}")

if LongLifespan not in [b for b in herb_age_test.__class__.__mro__ if b in herb_age_test.__class__._possible_mixins]:
    current_bases = list(herb_age_test.__class__.__mro__)
    original_concrete = Herbivore
    new_bases = tuple([LongLifespan] + [b for b in current_bases if b not in [Organism, ABC, object, LongLifespan]] )
    new_type = type("Herbivore_LongLifespan", new_bases, dict(herb_age_test.__class__.__dict__))
    herb_age_test.__class__ = new_type
    LongLifespan.__init__(herb_age_test)
    print(f"После добавления LongLifespan: {herb_age_test.__class__.__name__}, Max Age = {herb_age_test._max_age}")

--- Тестирование метода mutate для Herbivore ---
Начальное состояние: Herbivore ID: 0, Age: 0, Energy: 10, Population: 10, Alive: True (Max Age: 50)

--- Мутация Herbivore #1 ---
Организм 0 (Тип: Herbivore) не смог мутировать: нет Mixin'ов для удаления.
Текущее состояние: Herbivore ID: 0, Age: 0, Energy: 10, Population: 10, Alive: True (Max Age: 50)

--- Мутация Herbivore #2 ---
Организм 0 (Тип: Herbivore) не смог мутировать: нет Mixin'ов для удаления.
Текущее состояние: Herbivore ID: 0, Age: 0, Energy: 10, Population: 10, Alive: True (Max Age: 50)

--- Мутация Herbivore #3 ---
Организм 0 (Тип: Herbivore) не смог мутировать: нет Mixin'ов для удаления.
Текущее состояние: Herbivore ID: 0, Age: 0, Energy: 10, Population: 10, Alive: True (Max Age: 50)

--- Мутация Herbivore #4 ---


TypeError: __class__ assignment: 'Herbivore_HighFertilityMixin' object layout differs from 'Herbivore'

# 5. Собираем симуляцию вместе (5 баллов)


Напиши класс `Game`, который будет хранить список текущих животных и растений. Заполни его на своё усмотрение в начале игры (метод `__init__`), НО в начале не должно быть ни одного животного/растения с мутациями (mixin'ами).

У него должны быть:

- метод `__init__` (см. выше)

- метод `correct_age_energy`, который итерируется по всем живым организмам и делает с каждым следующие шаги:

1. Вычитает константу потребления энергии (они различны для животных и растений)
2. Прибавляет age на 1.
3. Если животное или растение стало слишком старым ИЛИ энергия стала <= 0, то оно умирает (и это логируется в поток вывода).


- метод `cycle_through_actions`, который итерируется по всем живым организмам и с каждым равновероятно делает одно из трёх действий: `replenish_energy` (для животных оно выбирает растение и уменьшает его популяцию в `PLANT_EATEN_COEF` и энергию в `PLANT_ENERGY_EATEN_COEF` раз, после чего повышает собственную энергию через `replenish_energy`), `reproduce`, `mutate`. Каждый из этих трёх действий должно корректно логироваться (чтобы мы понимали, что происходит в симуляции).


- метод `play_game(n)` - который запускает игру и делает последовательно `n` циклов жизни (запусков `correct_age_energy` и `cycle_through_actions`).

In [ ]:
# TODO

Собери все возможные коэффициенты и установи их на своё усмотрение в ячейке ниже.

Запусти игру. Какие результаты получились после запуска?)

In [ ]:
# TODO

Результаты

<b><font color="#FF69B4"> Ваш ответ здесь </font></b>